This script shows an example of a working spatiotemporal visualization that shows the following data:
- Acoustic (real dates, real locations)
- Aerial surveys (real dates, two fake locations)
- Zooplankton surveys (real dates, one fake location)

In [1]:
import altair as alt
import pandas as pd
import geopandas as gpd

In [2]:
concat3 = pd.read_csv('../data/concat3.csv',
                              parse_dates = ['between_days'])

concat3

,Unnamed: 0,between_days,depYear,c_uniqueUnitID,latitudeDeployed_DecDeg,longitudeDeployed_DecDeg,DataType
0,0,2011-02-17,2011,2011_BRP_CCB_S1016_Dep20_20110217_PU0205_FD020...,41.9412,-70.288,Acoustic
1,0,2011-02-18,2011,2011_BRP_CCB_S1016_Dep20_20110217_PU0205_FD020...,41.9412,-70.288,Acoustic
2,0,2011-02-19,2011,2011_BRP_CCB_S1016_Dep20_20110217_PU0205_FD020...,41.9412,-70.288,Acoustic
3,0,2011-02-20,2011,2011_BRP_CCB_S1016_Dep20_20110217_PU0205_FD020...,41.9412,-70.288,Acoustic
4,0,2011-02-21,2011,2011_BRP_CCB_S1016_Dep20_20110217_PU0205_FD020...,41.9412,-70.288,Acoustic
...,...,...,...,...,...,...,...
3735,64,2018-04-13,2018,NaN,41.9700,-70.430,Zooplankton
3736,65,2018-04-22,2018,NaN,41.9700,-70.430,Zooplankton
3737,66,2018-04-27,2018,NaN,41.9700,-70.430,Zooplankton
3738,67,2018-04-30,2018,NaN,41.9700,-70.430,Zooplankton


In [3]:
# Mass shapefile
land_shp = '../data/newengland/NEWENGLAND_POLY.shp'
land = gpd.read_file(land_shp).to_crs('epsg:4326')
land.head()

,FIPS,NAME,ACRES,SHAPE_AREA,SHAPE_LEN,geometry
0,33,NEW HAMPSHIRE,5935592.8,2.401324e+10,9.028755e+05,"POLYGON ((-71.08751 45.30145, -71.00860 44.282..."
1,0,VERMONT,6138697.6,2.486846e+10,8.630178e+05,"POLYGON ((-71.50476 45.01328, -71.52872 44.999..."
2,25,MASSACHUSETTS,5104241.5,2.063503e+10,1.442175e+06,"POLYGON ((-73.25806 42.74605, -73.01970 42.740..."
3,9,CONNECTICUT,3162420.2,1.279752e+10,6.018469e+05,"POLYGON ((-73.48423 42.04743, -73.04563 42.036..."
4,44,RHODE ISLAND,612942.3,2.477726e+09,3.326268e+05,"POLYGON ((-71.22898 41.70769, -71.23976 41.697..."


In [4]:
# Massachusetts plotted with altair
mass_land = land.loc[land['NAME']=='MASSACHUSETTS',:]

alt.Chart(mass_land).mark_geoshape(
    fill='#2a1d0c', stroke='#706545', strokeWidth=0.5
).project('mercator')

alt.Chart(...)

In [6]:
# full interactive visual -- acoustic + aerial + zooplankton

interval = alt.selection(type='interval', encodings=['x']) 
# interactive piece
    # different types
    # interactivity along x axis
# selections can have conditions

timeline_base = alt.Chart(concat3).mark_rect().encode(
    y = alt.Y('DataType:O', axis=alt.Axis(title='Data Type')),
    color = 'DataType:N'
).properties(
    width = 600
)

timeline_overview = timeline_base.encode(
    x = alt.X(
        'between_days:T', 
        timeUnit = 'yearmonthdate', 
        axis = alt.Axis(title='Date')
    )
).add_selection( # adding interactivity
    interval
).properties(
    height = 40
)

timeline_detail = timeline_base.encode(
    x = alt.X(
        'between_days:T', 
        timeUnit='yearmonthdate',
        axis = alt.Axis(title=''),
        scale = alt.Scale(domain=interval) # using the interactive selection to show X range
    )
).properties(
    height = 100
)

basemap = alt.Chart(mass_land).mark_geoshape(
    fill = 'lightgray', stroke='#706545', strokeWidth=0.5
).project('mercator').properties(
    width = 600,
    height = 300
)

points = alt.Chart(concat3).mark_point().encode(
    longitude = 'longitudeDeployed_DecDeg:Q',
    latitude = 'latitudeDeployed_DecDeg:Q',
    color = 'DataType:N'
).transform_filter(
    interval
).project("mercator").properties( # can put scale parameter
    width = 600,
    height = 300
)

March19 = alt.vconcat((basemap + points), timeline_detail, timeline_overview)


March19.save('CCB_SpatioTemporal_March19.html')

what do we need to see?
- the full temporal extent of each dataset
- which days the datasets overlap
- locations of detections

what outputs do we need?
- overlap days (reported number? csv list of dates? visual heatmap?)
- spatial proximity (how close is detection A to detection B?)